# 🗄️ DataSens E2 — Notebook 2 : Création du Schéma PostgreSQL

**🎯 Objectif** : Créer le schéma PostgreSQL complet (36 tables Merise E2) avec contraintes, index et données de référence

---

## 📋 Contenu de ce notebook

1. Rappel MCD/MLD (schémas Mermaid) - 36 tables E2
2. DDL PostgreSQL : CREATE TABLE pour les 36 tables E2
3. Index et contraintes (FK, UNIQUE, CHECK, ON DELETE)
4. Insertion des référentiels (type_donnee, type_meteo, type_indicateur, valence, type_emotion, etc.)
5. Corrections MPD optionnelles (référence vers scripts `tests/fix_mpd_*.sql` pour passer à 40 tables)
6. Contrôles : listes des tables, counts par table

---

## 🔒 RGPD & Gouvernance

⚠️ **Rappel important** :
- Pas de données personnelles directes (hash SHA-256 si nécessaire)
- Respect robots.txt pour le scraping
- Droits d'usage documentés par source

---

## ⚠️ Note sur les Sources

**Sources E1 testées** : Kaggle CSV, OpenWeatherMap API, RSS Multi-sources, Web Scraping (Vie-publique, data.gouv), GDELT GKG  
**Sources payantes/obsolètes** : Voir `docs/SOURCES_STATUS.md` pour liste complète (ex: NewsAPI payant $449/mois, SignalConso API obsolète)



## 🧭 Rappel MCD/MLD (Modèle Conceptuel/Logique de Données)

### 36 Tables cibles E2 (Architecture complète)

**Collecte** (4 tables) :
- `type_donnee`, `source`, `flux`, `archive_flux`

**Documents & Annotations** (9 tables) :
- `document`, `annotation`, `annotation_emotion`, `meta_annotation`, `emotion`, `type_emotion`, `valence`, `modele_ia`, `utilisateur`

**Géographie** (5 tables) :
- `pays`, `region`, `departement`, `commune`, `territoire`

**Contexte** (5 tables) :
- `type_meteo`, `meteo`, `type_indicateur`, `source_indicateur`, `indicateur`

**Thèmes & Événements** (5 tables) :
- `theme_category`, `theme`, `evenement`, `document_theme`, `document_evenement`

**Baromètres** (2 tables) :
- `source_barometre`, `document_baro`

**Pipeline & Qualité** (5 tables) :
- `pipeline`, `etape_etl`, `exec_etape`, `qc_rule`, `qc_result`

**Gouvernance** (2 tables) :
- `table_audit`, `table_version`

**Collecte** :
- `type_donnee` : Catégorisation des sources (Fichier, Base de données, API, Web Scraping, Big Data)
- `source` : Sources réelles (Kaggle, OpenWeatherMap, MonAvisCitoyen, etc.)
- `flux` : Traçabilité des collectes (date, format, manifest_uri)

**Corpus** :
- `document` : Documents bruts collectés (titre, texte, langue, hash_fingerprint)
- `territoire` : Géolocalisation (ville, code_insee, lat, lon)

**Contexte** :
- `type_meteo` : Types de conditions météo (clair, nuageux, pluie...)
- `meteo` : Relevés météo (température, humidité, pression, vent)
- `type_indicateur` : Types d'indicateurs (population, revenu, etc.)
- `source_indicateur` : Sources des indicateurs (INSEE, IGN...)
- `indicateur` : Valeurs d'indicateurs par territoire

**Thèmes/événements** :
- `theme` : Thèmes documentaires (politique, économie, environnement...)
- `evenement` : Événements temporels (date_event, avg_tone)
- `document_evenement` : Relation N-N documents ↔ événements

**Gouvernance pipeline** :
- `pipeline` : Description des pipelines ETL
- `etape_etl` : Étapes du pipeline avec ordre d'exécution

**Utilisateurs (trace)** :
- `utilisateur` : Utilisateurs du système (pour futures annotations)

**Qualité (min)** :
- `qc_rule` : Règles de contrôle qualité (placeholder)
- `qc_result` : Résultats des contrôles qualité (optionnel)

---

### Schéma Mermaid (simplifié)

```mermaid
erDiagram
    TYPE_DONNEE ||--o{ SOURCE : "a pour"
    SOURCE ||--o{ FLUX : "génère"
    FLUX ||--o{ DOCUMENT : "contient"
    TERRITOIRE ||--o{ DOCUMENT : "géolocalise"
    TERRITOIRE ||--o{ METEO : "mesure"
    TERRITOIRE ||--o{ INDICATEUR : "agrège"
    THEME ||--o{ EVENEMENT : "classe"
    DOCUMENT ||--o{ DOCUMENT_EVENEMENT : "refère"
    EVENEMENT ||--o{ DOCUMENT_EVENEMENT : "associe"
```



In [ ]:
# Imports et configuration
import os
from pathlib import Path

from dotenv import load_dotenv
from minio import Minio
from sqlalchemy import create_engine, text

# Charger .env
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == "notebooks" else NOTEBOOK_DIR
load_dotenv(PROJECT_ROOT / ".env")

# Connexion PostgreSQL
PG_HOST = os.getenv("POSTGRES_HOST", "localhost")
PG_PORT = int(os.getenv("POSTGRES_PORT", "5432"))
PG_DB = os.getenv("POSTGRES_DB", "datasens")
PG_USER = os.getenv("POSTGRES_USER", "ds_user")
PG_PASS = os.getenv("POSTGRES_PASS", "ds_pass")

PG_URL = f"postgresql+psycopg2://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}"
engine = create_engine(PG_URL, future=True)

# Configuration MinIO (DataLake)
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT", "http://localhost:9000")
MINIO_ACCESS_KEY = os.getenv("MINIO_ACCESS_KEY", "miniouser")
MINIO_SECRET_KEY = os.getenv("MINIO_SECRET_KEY", "miniosecret")
MINIO_BUCKET = os.getenv("MINIO_BUCKET", "datasens-raw")

print("🔌 Connexion PostgreSQL")
print("=" * 80)
print(f"📍 {PG_HOST}:{PG_PORT}/{PG_DB}")
print(f"👤 {PG_USER}")

# Test connexion PostgreSQL
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1"))
    print("✅ PostgreSQL : Connexion OK")

# Test connexion MinIO
try:
    minio_client = Minio(
        MINIO_ENDPOINT.replace("http://", "").replace("https://", ""),
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=MINIO_ENDPOINT.startswith("https")
    )

    # Fonction helper MinIO
    def ensure_bucket(bucket: str = MINIO_BUCKET):
        if not minio_client.bucket_exists(bucket):
            minio_client.make_bucket(bucket)

    def minio_upload(local_path: Path, dest_key: str) -> str:
        ensure_bucket(MINIO_BUCKET)
        minio_client.fput_object(MINIO_BUCKET, dest_key, str(local_path))
        return f"s3://{MINIO_BUCKET}/{dest_key}"

    ensure_bucket()
    print(f"✅ MinIO : DataLake OK → bucket: {MINIO_BUCKET}")
except Exception as e:
    print(f"⚠️ MinIO : Erreur connexion ({e}) - Vérifier docker-compose up -d")
    minio_client = None
    def minio_upload(local_path: Path, dest_key: str) -> str:
        return f"local://{local_path}"

print("\n✅ Configuration terminée !\n")


## 📐 DDL PostgreSQL : Création des 36 tables E2

Création des tables avec contraintes d'intégrité référentielle.  
**Ordre de création** : Respect des dépendances FK (référentiels → métier → liaisons).

**Note** : Les sources obsolètes/payantes ne sont **pas** implémentées.  
**Sources E1 testées** : Kaggle CSV, OpenWeatherMap API, RSS Multi-sources, Web Scraping (Vie-publique, data.gouv), GDELT GKG  
**Voir** `docs/SOURCES_STATUS.md` pour statut complet des sources.


In [ ]:
# DDL complet : 36 tables E2
# Basé sur MCD/MLD/MPD validés - Ordre respecte dépendances FK
# Chargement depuis fichier SQL pour lisibilité

ddl_file = PROJECT_ROOT / "tests" / "ddl_36_tables_e2.sql"

if ddl_file.exists():
    with open(ddl_file, encoding='utf-8') as f:
        ddl_sql = f.read()
    print(f"✅ DDL chargé depuis {ddl_file.name}")
else:
    print(f"⚠️ Fichier DDL non trouvé: {ddl_file}")
    print("   → Création DDL inline (18 tables E1 seulement)...")
    # Fallback DDL minimal (18 tables E1)
    ddl_sql = """
-- =====================================================
-- COLLECTE : Type de données, sources, flux
-- =====================================================

CREATE TABLE IF NOT EXISTS type_donnee (
  id_type_donnee SERIAL PRIMARY KEY,
  libelle VARCHAR(100) NOT NULL UNIQUE,
  description TEXT
);

CREATE TABLE IF NOT EXISTS source (
  id_source SERIAL PRIMARY KEY,
  id_type_donnee INT REFERENCES type_donnee(id_type_donnee) ON DELETE RESTRICT,
  nom VARCHAR(100) NOT NULL,
  url TEXT,
  fiabilite FLOAT CHECK (fiabilite >= 0 AND fiabilite <= 1)
);

CREATE TABLE IF NOT EXISTS flux (
  id_flux SERIAL PRIMARY KEY,
  id_source INT NOT NULL REFERENCES source(id_source) ON DELETE CASCADE,
  date_collecte TIMESTAMP NOT NULL DEFAULT NOW(),
  format VARCHAR(20),
  manifest_uri TEXT
);

-- =====================================================
-- CORPUS : Documents et territoires
-- =====================================================

CREATE TABLE IF NOT EXISTS territoire (
  id_territoire SERIAL PRIMARY KEY,
  ville VARCHAR(120),
  code_insee VARCHAR(10),
  lat FLOAT,
  lon FLOAT,
  CONSTRAINT unique_code_insee UNIQUE (code_insee)
);

CREATE TABLE IF NOT EXISTS document (
  id_doc SERIAL PRIMARY KEY,
  id_flux INT REFERENCES flux(id_flux) ON DELETE SET NULL,
  id_territoire INT REFERENCES territoire(id_territoire) ON DELETE SET NULL,
  titre TEXT,
  texte TEXT,
  langue VARCHAR(10),
  date_publication TIMESTAMP,
  hash_fingerprint VARCHAR(64) UNIQUE
);

-- =====================================================
-- CONTEXTE : Météo et indicateurs
-- =====================================================

CREATE TABLE IF NOT EXISTS type_meteo (
  id_type_meteo SERIAL PRIMARY KEY,
  code VARCHAR(20) UNIQUE NOT NULL,
  libelle VARCHAR(100) NOT NULL
);

CREATE TABLE IF NOT EXISTS meteo (
  id_meteo SERIAL PRIMARY KEY,
  id_territoire INT NOT NULL REFERENCES territoire(id_territoire) ON DELETE CASCADE,
  id_type_meteo INT REFERENCES type_meteo(id_type_meteo) ON DELETE SET NULL,
  date_obs TIMESTAMP NOT NULL,
  temperature FLOAT,
  humidite FLOAT CHECK (humidite >= 0 AND humidite <= 100),
  vent_kmh FLOAT CHECK (vent_kmh >= 0),
  pression FLOAT CHECK (pression > 0),
  meteo_type VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS type_indicateur (
  id_type_indic SERIAL PRIMARY KEY,
  code VARCHAR(50) UNIQUE NOT NULL,
  libelle VARCHAR(100),
  unite VARCHAR(20)
);

CREATE TABLE IF NOT EXISTS source_indicateur (
  id_source_indic SERIAL PRIMARY KEY,
  nom VARCHAR(100) NOT NULL,
  url TEXT
);

CREATE TABLE IF NOT EXISTS indicateur (
  id_indic SERIAL PRIMARY KEY,
  id_territoire INT NOT NULL REFERENCES territoire(id_territoire) ON DELETE CASCADE,
  id_type_indic INT NOT NULL REFERENCES type_indicateur(id_type_indic) ON DELETE RESTRICT,
  id_source_indic INT REFERENCES source_indicateur(id_source_indic) ON DELETE SET NULL,
  valeur FLOAT,
  annee INT CHECK (annee >= 1900 AND annee <= 2100)
);

-- =====================================================
-- THÈMES/ÉVÉNEMENTS
-- =====================================================

CREATE TABLE IF NOT EXISTS theme (
  id_theme SERIAL PRIMARY KEY,
  libelle VARCHAR(100) NOT NULL,
  description TEXT
);

CREATE TABLE IF NOT EXISTS evenement (
  id_event SERIAL PRIMARY KEY,
  id_theme INT REFERENCES theme(id_theme) ON DELETE SET NULL,
  date_event TIMESTAMP,
  avg_tone FLOAT CHECK (avg_tone >= -100 AND avg_tone <= 100),
  source_event VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS document_evenement (
  id_doc INT NOT NULL REFERENCES document(id_doc) ON DELETE CASCADE,
  id_event INT NOT NULL REFERENCES evenement(id_event) ON DELETE CASCADE,
  PRIMARY KEY (id_doc, id_event)
);

-- =====================================================
-- GOUVERNANCE PIPELINE
-- =====================================================

CREATE TABLE IF NOT EXISTS pipeline (
  id_pipeline SERIAL PRIMARY KEY,
  nom VARCHAR(100) NOT NULL,
  description TEXT,
  version VARCHAR(20)
);

CREATE TABLE IF NOT EXISTS etape_etl (
  id_etape SERIAL PRIMARY KEY,
  id_pipeline INT NOT NULL REFERENCES pipeline(id_pipeline) ON DELETE CASCADE,
  ordre INT NOT NULL CHECK (ordre > 0),
  nom_etape VARCHAR(100) NOT NULL,
  type_etape VARCHAR(20) CHECK (type_etape IN ('EXTRACT', 'TRANSFORM', 'LOAD')),
  description TEXT,
  CONSTRAINT unique_pipeline_ordre UNIQUE (id_pipeline, ordre)
);

-- =====================================================
-- UTILISATEURS (trace)
-- =====================================================

CREATE TABLE IF NOT EXISTS utilisateur (
  id_user SERIAL PRIMARY KEY,
  nom VARCHAR(100),
  role VARCHAR(50),
  organisation VARCHAR(100),
  date_creation TIMESTAMP DEFAULT NOW()
);

-- =====================================================
-- QUALITÉ (min)
-- =====================================================

CREATE TABLE IF NOT EXISTS qc_rule (
  id_qc_rule SERIAL PRIMARY KEY,
  nom_regle VARCHAR(100) NOT NULL,
  description TEXT,
  expression_sql TEXT
);

CREATE TABLE IF NOT EXISTS qc_result (
  id_qc_result SERIAL PRIMARY KEY,
  id_qc_rule INT REFERENCES qc_rule(id_qc_rule) ON DELETE CASCADE,
  id_flux INT REFERENCES flux(id_flux) ON DELETE CASCADE,
  date_check TIMESTAMP DEFAULT NOW(),
  statut VARCHAR(20) CHECK (statut IN ('PASS', 'FAIL', 'WARNING')),
  message TEXT
);
"""

print("📐 Exécution DDL PostgreSQL")
print("=" * 80)

# Option : Supprimer toutes les tables existantes avant de les recréer
# ⚠️ Décommenter DROP_TABLES = True si vous voulez reset complet (perte de données)
DROP_TABLES = True  # ⚠️ ACTIVÉ : Va supprimer toutes les tables existantes et les recréer

with engine.begin() as conn:
    if DROP_TABLES:
        print("⚠️ Suppression des tables existantes...")
        # Supprimer toutes les tables en cascade (attention : perte de données !)
        drop_sql = """
        DROP TABLE IF EXISTS qc_result CASCADE;
        DROP TABLE IF EXISTS qc_rule CASCADE;
        DROP TABLE IF EXISTS utilisateur CASCADE;
        DROP TABLE IF EXISTS etape_etl CASCADE;
        DROP TABLE IF EXISTS pipeline CASCADE;
        DROP TABLE IF EXISTS document_evenement CASCADE;
        DROP TABLE IF EXISTS evenement CASCADE;
        DROP TABLE IF EXISTS theme CASCADE;
        DROP TABLE IF EXISTS indicateur CASCADE;
        DROP TABLE IF EXISTS source_indicateur CASCADE;
        DROP TABLE IF EXISTS type_indicateur CASCADE;
        DROP TABLE IF EXISTS meteo CASCADE;
        DROP TABLE IF EXISTS type_meteo CASCADE;
        DROP TABLE IF EXISTS document CASCADE;
        DROP TABLE IF EXISTS territoire CASCADE;
        DROP TABLE IF EXISTS flux CASCADE;
        DROP TABLE IF EXISTS source CASCADE;
        DROP TABLE IF EXISTS type_donnee CASCADE;
        """
        conn.exec_driver_sql(drop_sql)
        print("✅ Tables supprimées")

    # Créer les tables (exécution séquentielle pour meilleure gestion d'erreurs)
    statements = [s.strip() for s in ddl_sql.split(';') if s.strip() and not s.strip().startswith('--')]
    for stmt in statements:
        if stmt:
            try:
                conn.exec_driver_sql(stmt + ';')
            except Exception as e:
                # Ignorer erreurs "already exists" pour IF NOT EXISTS
                if 'already exists' not in str(e).lower():
                    print(f"⚠️ Erreur création: {stmt[:50]}... → {e}")

print("✅ Tables E2 créées avec succès !")


## 🔗 Index et contraintes additionnelles

Création des index pour optimiser les requêtes (hash_fingerprint, dates, clés étrangères)


In [ ]:
# Index pour performance
indexes_sql = """
-- Index sur hash_fingerprint pour déduplication rapide
CREATE INDEX IF NOT EXISTS idx_document_hash_fingerprint ON document(hash_fingerprint);

-- Index sur dates pour requêtes temporelles
CREATE INDEX IF NOT EXISTS idx_document_date_publication ON document(date_publication);
CREATE INDEX IF NOT EXISTS idx_flux_date_collecte ON flux(date_collecte);
CREATE INDEX IF NOT EXISTS idx_meteo_date_obs ON meteo(date_obs);
CREATE INDEX IF NOT EXISTS idx_evenement_date_event ON evenement(date_event);

-- Index sur clés étrangères fréquentes
CREATE INDEX IF NOT EXISTS idx_document_id_flux ON document(id_flux);
CREATE INDEX IF NOT EXISTS idx_document_id_territoire ON document(id_territoire);
CREATE INDEX IF NOT EXISTS idx_flux_id_source ON flux(id_source);
CREATE INDEX IF NOT EXISTS idx_meteo_id_territoire ON meteo(id_territoire);
CREATE INDEX IF NOT EXISTS idx_indicateur_id_territoire ON indicateur(id_territoire);

-- Index composite pour recherche par territoire + date
CREATE INDEX IF NOT EXISTS idx_meteo_territoire_date ON meteo(id_territoire, date_obs DESC);
"""

print("🔗 Création des index")
print("=" * 80)

with engine.begin() as conn:
    conn.exec_driver_sql(indexes_sql)

print("✅ Index créés avec succès !")


## 📝 Insertion des référentiels

Insertion des données de référence nécessaires pour normaliser les données


In [ ]:
# Données de référence à insérer
referentiels = {
    "type_donnee": [
        ("Fichier plat", "CSV, JSON, Parquet..."),
        ("Base de données", "SQLite, PostgreSQL, MySQL..."),
        ("API", "REST API, GraphQL..."),
        ("Web Scraping", "HTML scraping, RSS..."),
        ("Big Data", "GDELT, fichiers volumineux..."),
    ],
    "type_meteo": [
        ("CLEAR", "Ciel clair"),
        ("CLOUDS", "Nuageux"),
        ("RAIN", "Pluie"),
        ("SNOW", "Neige"),
        ("THUNDERSTORM", "Orage"),
        ("FOG", "Brouillard"),
    ],
    "type_indicateur": [
        ("POPULATION", "Population totale", "habitants"),
        ("REVENU_MEDIAN", "Revenu médian", "€"),
        ("TAUX_CHOMAGE", "Taux de chômage", "%"),
        ("SUPERFICIE", "Superficie", "km²"),
    ],
    "source_indicateur": [
        ("INSEE", "https://www.insee.fr/"),
        ("IGN", "https://www.ign.fr/"),
        ("data.gouv.fr", "https://www.data.gouv.fr/"),
    ],
    "theme_category": [
        ("Société", "Thèmes liés à la société"),
        ("Politique", "Thèmes politiques"),
        ("Économie", "Thèmes économiques"),
        ("Environnement", "Thèmes environnementaux"),
        ("Santé", "Thèmes de santé"),
    ],
    "theme": [
        ("Politique", "Événements et analyses politiques"),
        ("Économie", "Actualités économiques"),
        ("Société", "Faits de société"),
        ("Environnement", "Écologie, climat, biodiversité"),
        ("Santé", "Santé publique, médical"),
        ("Sport", "Événements sportifs"),
        ("Culture", "Arts, spectacles, culture"),
        ("Technologie", "Innovation, numérique"),
    ],
    "valence": [
        ("Positive", "Émotions positives (joie, espoir, satisfaction)"),
        ("Neutre", "Émotions neutres (indifférence, calme)"),
        ("Negative", "Émotions négatives (colère, tristesse, peur)"),
    ],
    "type_emotion": [
        ("Joie", "Sentiment de bonheur", "Positive"),
        ("Colère", "Sentiment de frustration ou agressivité", "Negative"),
        ("Tristesse", "Sentiment de peine", "Negative"),
        ("Peur", "Sentiment d'anxiété", "Negative"),
        ("Espoir", "Sentiment d'optimisme", "Positive"),
        ("Neutre", "Pas d'émotion particulière", "Neutre"),
    ],
    "pays": [
        ("France",),
    ],
    "source_barometre": [
        ("INSEE Baromètre Social", "https://www.insee.fr/"),
        ("Data.gouv.fr", "https://www.data.gouv.fr/"),
    ],
    "qc_rule": [
        ("No duplicates", "Vérifier absence de doublons via hash_fingerprint", "SELECT COUNT(*) FROM document GROUP BY hash_fingerprint HAVING COUNT(*) > 1"),
        ("No NULL titles", "Tous les documents doivent avoir un titre", "SELECT COUNT(*) FROM document WHERE titre IS NULL"),
        ("Date range valid", "Les dates de publication doivent être raisonnables", "SELECT COUNT(*) FROM document WHERE date_publication < '1900-01-01' OR date_publication > NOW()"),
    ],
}

print("📝 Insertion des référentiels")
print("=" * 80)

with engine.begin() as conn:
    # Vérifier et corriger la structure de type_donnee si nécessaire
    try:
        # Vérifier si la colonne description existe
        result = conn.execute(text("""
            SELECT column_name
            FROM information_schema.columns
            WHERE table_name = 'type_donnee' AND column_name = 'description'
        """)).fetchone()

        if not result:
            # Ajouter la colonne description si elle n'existe pas
            print("⚠️ Colonne 'description' manquante dans type_donnee, ajout en cours...")
            conn.execute(text("ALTER TABLE type_donnee ADD COLUMN IF NOT EXISTS description TEXT"))
            print("✅ Colonne 'description' ajoutée")

        # Vérifier si la contrainte UNIQUE sur libelle existe
        constraint_exists = conn.execute(text("""
            SELECT 1
            FROM information_schema.table_constraints
            WHERE table_name = 'type_donnee'
              AND constraint_type = 'UNIQUE'
              AND constraint_name LIKE '%libelle%'
        """)).fetchone()

        if not constraint_exists:
            # Vérifier si un index unique existe
            index_exists = conn.execute(text("""
                SELECT 1
                FROM pg_indexes
                WHERE tablename = 'type_donnee'
                  AND indexdef LIKE '%libelle%'
                  AND indexdef LIKE '%UNIQUE%'
            """)).fetchone()

            if not index_exists:
                print("⚠️ Contrainte UNIQUE manquante sur libelle, ajout en cours...")
                conn.execute(text("ALTER TABLE type_donnee ADD CONSTRAINT type_donnee_libelle_unique UNIQUE (libelle)"))
                print("✅ Contrainte UNIQUE sur libelle ajoutée")
    except Exception as e:
        print(f"⚠️ Vérification structure: {e}")

    # type_donnee - Insertion avec gestion robuste des conflits
    inserted_count = 0
    for libelle, desc in referentiels["type_donnee"]:
        try:
            # Essayer d'abord avec ON CONFLICT
            result = conn.execute(text("""
                INSERT INTO type_donnee (libelle, description)
                VALUES (:libelle, :desc)
                ON CONFLICT (libelle) DO NOTHING
                RETURNING id_type_donnee
            """), {"libelle": libelle, "desc": desc})
            if result.scalar():
                inserted_count += 1
        except Exception:
            # Si ON CONFLICT échoue, vérifier si l'entrée existe déjà
            existing = conn.execute(text("""
                SELECT id_type_donnee
                FROM type_donnee
                WHERE libelle = :libelle
            """), {"libelle": libelle}).fetchone()
            if not existing:
                # Si n'existe pas, insérer sans ON CONFLICT
                conn.execute(text("""
                    INSERT INTO type_donnee (libelle, description)
                    VALUES (:libelle, :desc)
                """), {"libelle": libelle, "desc": desc})
                inserted_count += 1

    print(f"✅ type_donnee : {inserted_count} entrées insérées (total: {len(referentiels['type_donnee'])})")

    # type_meteo
    for code, libelle in referentiels["type_meteo"]:
        conn.execute(text("""
            INSERT INTO type_meteo (code, libelle)
            VALUES (:code, :libelle)
            ON CONFLICT (code) DO NOTHING
        """), {"code": code, "libelle": libelle})
    print(f"✅ type_meteo : {len(referentiels['type_meteo'])} entrées")

    # type_indicateur
    for code, libelle, unite in referentiels["type_indicateur"]:
        conn.execute(text("""
            INSERT INTO type_indicateur (code, libelle, unite)
            VALUES (:code, :libelle, :unite)
            ON CONFLICT (code) DO NOTHING
        """), {"code": code, "libelle": libelle, "unite": unite})
    print(f"✅ type_indicateur : {len(referentiels['type_indicateur'])} entrées")

    # source_indicateur
    for nom, url in referentiels["source_indicateur"]:
        conn.execute(text("""
            INSERT INTO source_indicateur (nom, url)
            VALUES (:nom, :url)
            ON CONFLICT DO NOTHING
        """), {"nom": nom, "url": url})
    print(f"✅ source_indicateur : {len(referentiels['source_indicateur'])} entrées")

    # theme
    for libelle, desc in referentiels["theme"]:
        conn.execute(text("""
            INSERT INTO theme (libelle, description)
            VALUES (:libelle, :desc)
            ON CONFLICT DO NOTHING
        """), {"libelle": libelle, "desc": desc})
    print(f"✅ theme : {len(referentiels['theme'])} entrées")

    # qc_rule
    for nom, desc, expr in referentiels["qc_rule"]:
        try:
            conn.execute(text("""
                INSERT INTO qc_rule (nom_regle, description, expression_sql)
                VALUES (:nom, :desc, :expr)
                ON CONFLICT DO NOTHING
            """), {"nom": nom, "desc": desc, "expr": expr})
        except Exception:
            # Si colonne expression_sql n'existe pas (E2), utiliser colonnes E2
            conn.execute(text("""
                INSERT INTO qc_rule (code, libelle, definition)
                VALUES (:nom, :desc, :expr)
                ON CONFLICT (code) DO NOTHING
            """), {"nom": nom.lower().replace(' ', '_'), "desc": nom, "expr": desc})
    print(f"✅ qc_rule : {len(referentiels['qc_rule'])} entrées")

    # Nouveaux référentiels E2
    if "valence" in referentiels:
        for label, desc in referentiels["valence"]:
            conn.execute(text("""
                INSERT INTO valence (label, description)
                VALUES (:label, :desc)
                ON CONFLICT (label) DO NOTHING
            """), {"label": label, "desc": desc})
        print(f"✅ valence : {len(referentiels['valence'])} entrées")

    if "type_emotion" in referentiels:
        for libelle, desc, valence_label in referentiels["type_emotion"]:
            id_valence = conn.execute(text("SELECT id_valence FROM valence WHERE label = :label"), {"label": valence_label}).scalar()
            if id_valence:
                conn.execute(text("""
                    INSERT INTO type_emotion (id_valence, libelle, description)
                    VALUES (:id_valence, :libelle, :desc)
                    ON CONFLICT (libelle) DO NOTHING
                """), {"id_valence": id_valence, "libelle": libelle, "desc": desc})
        print(f"✅ type_emotion : {len(referentiels['type_emotion'])} entrées")

    if "pays" in referentiels:
        for nom in referentiels["pays"]:
            conn.execute(text("""
                INSERT INTO pays (nom)
                VALUES (:nom)
                ON CONFLICT (nom) DO NOTHING
            """), {"nom": nom})
        print(f"✅ pays : {len(referentiels['pays'])} entrées")

    if "theme_category" in referentiels:
        for libelle, desc in referentiels["theme_category"]:
            conn.execute(text("""
                INSERT INTO theme_category (libelle, description)
                VALUES (:libelle, :desc)
                ON CONFLICT DO NOTHING
            """), {"libelle": libelle, "desc": desc})
        print(f"✅ theme_category : {len(referentiels['theme_category'])} entrées")

    if "source_barometre" in referentiels:
        for nom, url in referentiels["source_barometre"]:
            conn.execute(text("""
                INSERT INTO source_barometre (nom, url)
                VALUES (:nom, :url)
                ON CONFLICT DO NOTHING
            """), {"nom": nom, "url": url})
        print(f"✅ source_barometre : {len(referentiels['source_barometre'])} entrées")

    # theme avec FK vers theme_category
    if "theme" in referentiels and "theme_category" in referentiels:
        for libelle, desc in referentiels["theme"]:
            # Trouver une catégorie par défaut
            id_cat = conn.execute(text("SELECT id_theme_cat FROM theme_category LIMIT 1")).scalar()
            if id_cat:
                conn.execute(text("""
                    INSERT INTO theme (id_theme_cat, libelle, description)
                    VALUES (:id_cat, :libelle, :desc)
                    ON CONFLICT DO NOTHING
                """), {"id_cat": id_cat, "libelle": libelle, "desc": desc})
        print(f"✅ theme (E2) : {len(referentiels['theme'])} entrées")

print("\n✅ Tous les référentiels insérés !")


## ✅ Contrôles : Vérification des tables créées

Liste des tables et comptage des entrées par table


In [ ]:
import pandas as pd

print("📊 Liste des tables PostgreSQL")
print("=" * 80)

# Lister toutes les tables
query_tables = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
  AND table_type = 'BASE TABLE'
ORDER BY table_name;
"""

df_tables = pd.read_sql(query_tables, engine)
print(f"\n✅ {len(df_tables)} tables créées :\n")
for table in df_tables['table_name']:
    print(f"   • {table}")

# Compter les entrées par table
print("\n📈 Nombre d'entrées par table :")
print("=" * 80)

counts = {}
for table in df_tables['table_name']:
    try:
        count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", engine).iloc[0]['count']
        counts[table] = count
    except Exception as e:
        counts[table] = f"Erreur: {e}"

df_counts = pd.DataFrame(list(counts.items()), columns=['Table', 'Count'])
print(df_counts.to_string(index=False))

print("\n✅ Schéma PostgreSQL E2 créé avec succès !")
print(f"   📊 {len(df_tables)} tables créées")
print("\n💡 **Corrections MPD optionnelles** :")
print("   Pour passer à 40 tables avec tables de liaison N-N, exécutez :")
print("   • tests/fix_mpd_complete.sql (script maître)")
print("   • Voir tests/README_MPD_FIXES.md pour détails")
print("\n   ➡️ Passez au notebook 03_ingest_sources.ipynb")
